In [88]:
pip install pandas

In [ ]:
pip install openpyxl

In [ ]:
from konlpy.tag import Kkma

In [1]:
kkma = Kkma()

NameError: name 'Kkma' is not defined

In [89]:
import pandas as pd

In [4]:
# 파일명
xls = pd.ExcelFile('pencil_tree_verb_adj.xlsx')

# Daraframe형식으로 엑셀 파일 읽기
df = pd.read_excel(xls, engine='openpyxl')

In [99]:
from collections import defaultdict, Counter

freq_dict = defaultdict(dict)


for idx, row in df.iterrows():
    item, group = row["Item"], row["group"]
    vv, vv2, vv3 = row["VV"], row["VV2"], row["VV3"]
    va, va2 = row["VA"], row["VA2"]

    #초기화 코드
    if not freq_dict[item]:
        freq_dict[item]["Verb"] = {
            "Young": [],
            "old": []
        }
        freq_dict[item]["Adj"] = {
            "Young": [],
            "old": []
        }

    #작업 코드
    if vv and not isinstance(vv, float):
        freq_dict[item]["Verb"][group].append(vv)
    if vv2 and not isinstance(vv2, float):
        freq_dict[item]["Verb"][group].append(vv2)
    if vv3 and not isinstance(vv3, float):
        freq_dict[item]["Verb"][group].append(vv3)
    if va and not isinstance(va, float):
        freq_dict[item]["Adj"][group].append(va)
    if va2 and not isinstance(va2, float):
        freq_dict[item]["Adj"][group].append(va2)

# 카운터 객체 적용
for item, tag_dict in freq_dict.items():
    for tag, group_dict in tag_dict.items():
        for group, words in group_dict.items():
            freq_dict[item][tag][group] = Counter(words)

print(freq_dict)


defaultdict(<class 'dict'>, {'Scissors': {'Verb': {'Young': Counter({'자르다': 55, '사용하다': 10, '있다': 9, '들다': 5, '오리다': 5, '사다': 5, '가져다주다': 3, '가지다': 2, '이다': 2, '주다': 2, '가져오다': 2, '편리하다': 1, '무섭다': 1, '챙기다': 1, '두다': 1, '유용하다': 1, '붙다': 1, '저질이다': 1, '꺼내다': 1, '빛나다': 1, '쓰다': 1, '보다': 1, '뜯다': 1, '이용하다': 1, '베이다': 1}), 'old': Counter({'자르다': 36, '가지다': 14, '오리다': 8, '사용하다': 5, '필요하다': 4, '만들다': 4, '들다2': 4, '찾다': 3, '쓰다2': 3, '하다': 3, '부러지다': 2, '있다': 2, '썰다': 2, '재단하다': 2, '가져오다': 2, '빌리다': 1, '베이다': 1, '위험하다': 1, '구하다': 1, '가지고': 1, '짓다': 1, '쓰다': 1, '사다': 1, '가져다주다': 1, '이용하다': 1, '이다': 1, '주다': 1})}, 'Adj': {'Young': Counter({'날카롭다': 2, '없다': 2, '낫다': 1}), 'old': Counter({'좋다': 1, '좋아하다': 1, '많다': 1})}}, 'Ruler': {'Verb': {'Young': Counter({'재다': 36, '긋다': 10, '필요하다': 10, '사용하다': 10, '쓰다': 8, '대다': 5, '있다': 5, '맞다': 3, '유용하다': 2, '때리다': 2, '되다': 2, '사다': 2, '이용하다': 2, '가져오다': 1, '보이다': 1, '뜯다': 1, '구매하다': 1, '측정하다': 1, '빌려주다': 1, '다양하다': 1, '이다': 1, '잇다': 1, '구입다하다': 1, '두고오다': 1, 

In [102]:
def create_df_from_counter(counter, key_tag):
    total = counter.total()
    words = [word for word, cnt in counter.most_common()]
    fre = [cnt for word, cnt in counter.most_common()]
    percent = [v / total * 100 for v in fre]

    return pd.DataFrame({
        key_tag: words,
        "fre": fre,
        "percent": percent
    })

# 가로 병합
def concat_horizontal(item, tag_dict):
    vy, vo, ay, ao = tag_dict["Verb"]["Young"], tag_dict["Verb"]["old"], tag_dict["Adj"]["Young"], tag_dict["Adj"]["old"]

    s1= pd.Series([item], name="Item")
    gn_s1 = pd.Series(["Young"], name="Group")
    gn_s2 = pd.Series(["old"], name="Group")

    df1 = create_df_from_counter(vy, "Verb")
    df2 = create_df_from_counter(vo, "Verb")
    df3 = create_df_from_counter(ay, "Adjective")
    df4 = create_df_from_counter(ao, "Adjective")

    result = pd.concat([s1, gn_s1, df1, gn_s2, df2, gn_s1, df3, gn_s2, df4], axis=1)

    return result

# 세로 병합
def concat_vertical(df_list):
    return pd.concat(df_list, ignore_index=True)


def process(freq_dict):
    df_list = [concat_horizontal(item, tag_dict) for item, tag_dict in freq_dict.items()]
    return concat_vertical(df_list)

In [103]:
result = process(freq_dict)

In [104]:
# 엑셀 생성
import os

dir_name = "freq_result"
file_name = "freq_result"

if not os.path.exists(dir_name):
    os.mkdir(dir_name)

pd.DataFrame(result).to_excel(f'{dir_name}/{file_name}.xlsx', index=False)